In [ ]:
from azure.storage.blob import BlobServiceClient
import pandas as pd
from io import StringIO

# Ta connection string
connection_string = ""

# Nom du container
container_name = "76byj86oc9kf"
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Récupère le container
container_client = blob_service_client.get_container_client(container_name)

In [ ]:
# Liste des blobs
blobs_list = container_client.list_blobs()

# Stockage des DataFrames
df_list = []

for blob in blobs_list:
    if blob.name.startswith("transactions_") and blob.name.endswith(".csv"):
        blob_client = container_client.get_blob_client(blob.name)
        blob_data = blob_client.download_blob().readall()
        
        df = pd.read_csv(StringIO(blob_data.decode('utf-8')), sep=';', comment='#')
        
        expected_cols = ['transaction_id', 'client_id', 'date', 'hour', 'minute', 'product_id', 'quantity', 'store_id']
        
        missing_cols = [col for col in expected_cols if col not in df.columns]
        if missing_cols:
            print(f"Attention, colonnes manquantes dans {blob.name} : {missing_cols}")
            continue
        
        df = df[expected_cols]
        df['source_file'] = blob.name
        df_list.append(df)

In [24]:
df_all_transactions.head()

,transaction_id,client_id,date,hour,minute,product_id,quantity,store_id,source_file
0,6546228973719136,30491813828527,2023-11-23,11,49,341156340361391,2,4279246846108715,transactions_2023-11-23_11.csv
1,4288635687326247,341196857404028,2023-11-23,11,10,4393920951705710,3,4187214098379,transactions_2023-11-23_11.csv
2,30418179475593,180058235768439,2023-11-23,11,7,3565073637977222,3,4044611508927039,transactions_2023-11-23_11.csv
3,676115624733,4774906633038108,2023-11-23,11,36,4557030505164910884,5,3561934145164427,transactions_2023-11-23_11.csv
4,561040125214,344078070286076,2023-11-23,11,2,347751851467464,2,676247665430,transactions_2023-11-23_11.csv


In [48]:
# Fusion des fichiers
if df_list:
    df_all_transactions = pd.concat(df_list, ignore_index=True)
    print(f"{len(df_list)} fichiers chargés")
    print(df_all_transactions.head())
else:
    print("Aucun fichier de transaction trouvé.")
    
print(df_list)

101 fichiers chargés
     transaction_id         client_id        date  hour  minute  \
0  6546228973719136    30491813828527  2023-11-23    11      49   
1  4288635687326247   341196857404028  2023-11-23    11      10   
2    30418179475593   180058235768439  2023-11-23    11       7   
3      676115624733  4774906633038108  2023-11-23    11      36   
4      561040125214   344078070286076  2023-11-23    11       2   

            product_id  quantity          store_id  \
0      341156340361391         2  4279246846108715   
1     4393920951705710         3     4187214098379   
2     3565073637977222         3  4044611508927039   
3  4557030505164910884         5  3561934145164427   
4      347751851467464         2      676247665430   

                      source_file  
0  transactions_2023-11-23_11.csv  
1  transactions_2023-11-23_11.csv  
2  transactions_2023-11-23_11.csv  
3  transactions_2023-11-23_11.csv  
4  transactions_2023-11-23_11.csv  
[            transaction_id        

In [53]:
def extract_date_from_source_file(file_name):
    match = re.search(r'transactions_(\d{4}-\d{2}-\d{2})_\d+\.csv', file_name)
    return match.group(1) if match else None

# Appliquer la correction uniquement sur les lignes où date == "1999-01-01"
mask = df_all_transactions['date'] == '1999-01-01'

# Appliquer l'extraction et la mise à jour de la date
df_all_transactions.loc[mask, 'date'] = df_all_transactions.loc[mask, 'source_file'].apply(extract_date_from_source_file)

In [54]:
display(df_all_transactions[df_all_transactions["transaction_id"]==639054617217	])

,transaction_id,client_id,date,hour,minute,product_id,quantity,store_id,source_file,id,timestamp
2003413,639054617217,4230855715446,2023-11-30,9,30,2703290632160696,4,2227038929805094,transactions_2023-11-30_9.csv,6390546172174230855715446,1999-01-01 09:30:00


In [35]:
doublons = df_all_transactions[df_all_transactions.duplicated(subset=['transaction_id'], keep=False)]

In [ ]:
doublons = df_all_transactions[df_all_transactions.duplicated(subset=['transaction_id','client_id'], keep=False)]

In [31]:
print(df_all_transactions[df_all_transactions['transaction_id'] == 503871503143])


         transaction_id         client_id        date  hour  minute  \
2860       503871503143  6011665636969275  2023-11-23    11      51   
1604348    503871503143   213117691987320  2023-11-29    15      26   

               product_id  quantity          store_id  \
2860        4335151178742         2  4279246846108715   
1604348  4370500560740584         3  2227038929805094   

                            source_file  
2860     transactions_2023-11-23_11.csv  
1604348  transactions_2023-11-29_15.csv  


In [36]:
display(doublons.count())

transaction_id    146
client_id         146
date              146
hour              146
minute            146
product_id        146
quantity          146
store_id          146
source_file       146
dtype: int64

In [50]:
df_all_transactions['id'] = df_all_transactions['transaction_id'].astype(str) + df_all_transactions['client_id'].astype(str)
df_all_transactions['timestamp'] = pd.to_datetime(
    df_all_transactions['date'].astype(str) + ' ' +
    df_all_transactions['hour'].astype(str).str.zfill(2) + ':' +
    df_all_transactions['minute'].astype(str).str.zfill(2) + ':00',
    format='%Y-%m-%d %H:%M:%S'
)



In [1]:
import pandas as pd
from io import StringIO
from datetime import datetime
from zoneinfo import ZoneInfo
import os

from google.cloud import bigquery

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "gbqkey/inthememory-d591655b1691.json"

client = bigquery.Client()

In [12]:
 query = """
        SELECT 
            id as clientId,
            account_id,
        FROM `inthememory.Silver.clients`
        WHERE isEnabled = true
    """

query_job = client.query(query)  # lance la requête
df_silver_clients = query_job.result().to_dataframe()  # récupère le résultat dans un DataFrame pandas

In [ ]:
df_all_transactions = df_all_transactions.merge(
    df_silver_clients,
    left_on='client_id',
    right_on='clientId',
    how='left'
)


In [ ]:
df_transactions_silver = df_all_transactions[['id','transaction_id','client_id','account_id','product_id','store_id','quantity','timestamp','source_file']].copy()

df_transactions_silver['client_id'] = df_transactions_silver['client_id'].astype(str)
df_transactions_silver['transaction_id'] = df_transactions_silver['transaction_id'].astype(str)
df_transactions_silver['product_id'] = df_transactions_silver['product_id'].astype(str)
df_transactions_silver['store_id'] = df_transactions_silver['store_id'].astype(str)

In [46]:
display(df_transactions_silver.head())

,id,transaction_id,client_id,product_id,store_id,quantity,timestamp,source_file,clientId,account_id
0,654622897371913630491813828527,6546228973719136,30491813828527,341156340361391,4279246846108715,2,2023-11-23 11:49:00,transactions_2023-11-23_11.csv,30491813828527,4192787940932
1,4288635687326247341196857404028,4288635687326247,341196857404028,4393920951705710,4187214098379,3,2023-11-23 11:10:00,transactions_2023-11-23_11.csv,341196857404028,2376786084840541
2,30418179475593180058235768439,30418179475593,180058235768439,3565073637977222,4044611508927039,3,2023-11-23 11:07:00,transactions_2023-11-23_11.csv,180058235768439,4092136218742784
3,6761156247334774906633038108,676115624733,4774906633038108,4557030505164910884,3561934145164427,5,2023-11-23 11:36:00,transactions_2023-11-23_11.csv,4774906633038108,213189878309297
4,561040125214344078070286076,561040125214,344078070286076,347751851467464,676247665430,2,2023-11-23 11:02:00,transactions_2023-11-23_11.csv,344078070286076,213101308568228


In [ ]:
display(df_transactions_silver.head())

In [ ]:
transactions_2023-11-23_11.csv

In [18]:
import re


In [28]:
extract_datetime_from_filename('transactions_2023-11-23_8.csv')

In [27]:
extract_datetime_from_filename('transactions_2023-11-23_11.csv')

datetime.datetime(2023, 11, 23, 11, 0)

In [ ]:
def extract_datetime_from_filename(filename):
    match = re.search(r'transactions_(\d{4}-\d{2}-\d{2})_(\d{1,2})\.csv', filename)
    if match:
        date_str = match.group(1)
        hour_str = match.group(2)
        return datetime.strptime(f"{date_str} {hour_str}:00:00", "%Y-%m-%d %H:%M:%S")
    return None



In [31]:
target_table = "inthememory.Gold.transactions"
dataset_id, table_id = target_table.split(".")[1], target_table.split(".")[2]

table_exists = True
try:
    table_ref = client.dataset(dataset_id).table(table_id)
    client.get_table(table_ref)
except Exception:
    table_exists = False
table_exists

True

In [45]:
query = """
        SELECT 
        *
        FROM `inthememory.Silver.transactions`
        WHERE id NOT IN (SELECT transactionIdUnique from `inthememory.Gold.transactions`)
        """
query_job = client.query(query)  # lance la requête
df = query_job.result().to_dataframe()  
display(df)

,id,transaction_id,client_id,account_id,product_id,store_id,quantity,timestamp,source_file
0,6390546172174230855715446,639054617217,4230855715446,30291250339628,2703290632160696,2227038929805094,4,1999-01-01 09:30:00,transactions_2023-11-30_9.csv
1,3572620932010496378867883482990,3572620932010496,378867883482990,213153064075641,3583173127081762,4910033669017,2,1999-01-01 08:27:00,transactions_2023-11-30_8.csv
2,304603009191642707924213773750,30460300919164,2707924213773750,345042307226973,3555368637760418,675970708342,3,1999-01-01 09:04:00,transactions_2023-11-30_9.csv
3,35659112436205553506005157191000,3565911243620555,3506005157191000,3529699975951042,6011025079172087,4397925116705733,5,1999-01-01 17:31:00,transactions_2023-11-30_17.csv
4,35298314722389183521763675403171,3529831472238918,3521763675403171,30111040129594,2290786470146522,3541084316409501,5,1999-01-01 11:37:00,transactions_2023-11-30_11.csv
...,...,...,...,...,...,...,...,...,...
2573,4268598002151180076034273328,4268598002151,180076034273328,4695989339562960500,2293861137257967,4168704593076572910,1,1999-01-01 13:56:00,transactions_2023-11-30_13.csv
2574,180018094273939180064754218103,180018094273939,180064754218103,6540361785636549,4478260060923500,3582615269592346,3,1999-01-01 20:13:00,transactions_2023-11-30_20.csv
2575,3771088872260133581938264176902,377108887226013,3581938264176902,676342474837,6536195755696026,4788063003775978,4,1999-01-01 18:41:00,transactions_2023-11-30_18.csv
2576,48319163608021134773526424431417139,4831916360802113477,3526424431417139,4313898119780464802,4767005399478498,3561590953381915,5,1999-01-01 09:41:00,transactions_2023-11-30_9.csv


In [42]:
if table_exists:
        query_max_timestamp = """
        SELECT 
            MAX(timestamp) as max_timestamp
        FROM `inthememory.Gold.transactions`
        """
        query_job = client.query(query_max_timestamp)  # lance la requête
        df_max = query_job.result().to_dataframe()  
        max_timestamp = df_max['max_timestamp'].iloc[0]



query = f"""
        SELECT 
            id as transactionIdUnique,
            transaction_id as transactionId, 
            client_id as clientId,
            account_id as accountId,
            product_id as productId,
            store_id as storeId,
            quantity,
            timestamp
        FROM `inthememory.Silver.transactions`
        WHERE timestamp> '{max_timestamp}'
        """
        
query_job = client.query(query)  # lance la requête
df_gold_transactions = query_job.result().to_dataframe()  # récupère le résultat dans un DataFrame pandas

In [ ]:
target_table = "inthememory.Gold.transactions"
dataset_id, table_id = target_table.split(".")[1], target_table.split(".")[2]

    # Vérifie si la table existe
    table_exists = True
    try:
        table_ref = client.dataset(dataset_id).table(table_id)
        client.get_table(table_ref)
    except Exception:
        table_exists = False
    
    if table_exists:
        query_max_timestamp = """
        SELECT 
            MAX(timestamp) as max_timestamp
        FROM `inthememory.Gold.transactions`
        """
        query_job = client.query(query_max_timestamp)  # lance la requête
        df_max = query_job.result().to_dataframe()  
        max_timestamp = df_max['max_timestamp'].iloc[0]